In [2]:
!pip install concreteproperties

import numpy as np
from concreteproperties.material import Concrete, SteelBar
import concreteproperties.stress_strain_profile as ssp
from sectionproperties.pre.library.primitive_sections import (
    rectangular_section,
    circular_section,
)
from concreteproperties.pre import add_bar_rectangular_array
from concreteproperties.concrete_section import ConcreteSection
from concreteproperties.results import MomentCurvatureResults

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 96.5/96.5 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 225.4 kB/s eta 0:00:00
     ------------------------------------ 205.2/205.2 kB 211.5 kB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 247.6 kB/s eta 0:00:00
     ------------------------------------ 204.9/204.9 kB 166.1 kB/s eta 0:00:00
     --------------------------------------- 54.2/54.2 kB 93.7 kB/s eta 0:00:00
     ------------------------------------ 147.6/147.6 kB 120.5 kB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 207.9 kB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.0
    Uninstalling shapely-2.0.0:
      Successfully uninstalled shapely-2.0.0


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
conc_linear = Concrete(
    name="Linear Concrete",
    density=2.4e-6,
    stress_strain_profile=ssp.ConcreteLinear(
        elastic_modulus=35e3, ultimate_strain=0.0035
    ),
    ultimate_stress_strain_profile=ssp.BilinearStressStrain(
        compressive_strength=40,
        compressive_strain=0.00175,
        ultimate_strain=0.0035,
    ),
    flexural_tensile_strength=3.5,
    colour="lightgrey",
)

conc_linear_no_tension = Concrete(
    name="Linear Concrete (No T)",
    density=2.4e-6,
    stress_strain_profile=ssp.ConcreteLinearNoTension(
        elastic_modulus=35e3, ultimate_strain=0.0035
    ),
    ultimate_stress_strain_profile=ssp.BilinearStressStrain(
        compressive_strength=40,
        compressive_strain=0.00175,
        ultimate_strain=0.0035,
    ),
    flexural_tensile_strength=3.5,
    colour="lightgrey",
)


conc_nonlinear = Concrete(
    name="Non-Linear Concrete",
    density=2.4e-6,
    stress_strain_profile=ssp.EurocodeNonLinear(
        elastic_modulus=35e3,
        ultimate_strain=0.0035,
        compressive_strength=40,
        compressive_strain=0.0023,
        tensile_strength=3.5,
        tension_softening_stiffness=10e3,
    ),
    ultimate_stress_strain_profile=ssp.BilinearStressStrain(
        compressive_strength=40,
        compressive_strain=0.00175,
        ultimate_strain=0.0035,
    ),
    flexural_tensile_strength=3.5,
    colour="lightgrey",
)

conc_material_list = [
    conc_linear,
    conc_linear_no_tension,
    conc_nonlinear,
]

steel_ep = SteelBar(
    name="Steel - Elastic-Plastic",
    density=7.85e-6,
    stress_strain_profile=ssp.SteelElasticPlastic(
        yield_strength=500,
        elastic_modulus=200e3,
        fracture_strain=0.05,
    ),
    colour="grey",
)

steel_hd = SteelBar(
    name="Steel - Hardening",
    density=7.85e-6,
    stress_strain_profile=ssp.SteelHardening(
        yield_strength=500,
        elastic_modulus=200e3,
        fracture_strain=0.05,
        ultimate_strength=600,
    ),
    colour="grey",
)

steel_material_list = [
    steel_ep,
    steel_hd,
]

### Point 1 : $\phi$ at cracking (just before cracking)

In [27]:
depth_pt_one = h/2
area_steel_pt_one = area_steel
# total = area_gross + area_steel_pt_one

point_arr = [
    ["concrete",area_gross,depth_pt_one],
    ["reinforcement steel",area_steel_pt_one,d],
#     ["total",total,0]
]

length = len(point_arr)
# print(length)

top_value = 0
total = 0

i = 0
while i < length:
    if point_arr[i][0] == "concrete":
        multi = 1
    else:
        multi = n_conc
    top_value += point_arr[i][1]*point_arr[i][2]*multi
    total += point_arr[i][1]
    i += 1

ct = top_value/total

point_arr.append(["total",total,""])

Icr = beam_internia + area_gross*(depth_pt_one - ct) + (n_conc - 1)*area_steel_pt_one*math.pow(d - ct,2)


## Applying the Varignon Theroem!
df = pd.DataFrame(point_arr
                 ,columns=["Material", "Area (sq.mm)","Centroid measured from the top"])
print(df)
print('======================')
print('ct =  ' + str(ct))
print('I_uncracked =  ' + str(Icr))

              Material  Area (sq.mm) Centroid measured from the top
0             concrete        150000                          250.0
1  reinforcement steel          1935                            450
2                total        151935                               
ct =  292.66462632046597
I_uncracked =  3453898922.3902807
